In [102]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import pandas as pd

In [103]:
df = pd.read_csv('testing_tfids.csv')

In [104]:
X = df.drop(['Unnamed: 0','author', 'is_bot'], axis=1)
y = df['is_bot']

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [106]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

def pipe_maker(classifier):
    pipe = Pipeline([('clf', classifier)])
    return pipe

In [107]:
def gridsearch_maker(pipeline, params):
    grid = GridSearchCV(pipeline, param_grid=params, cv=5, scoring='f1_weighted')
    return grid

In [108]:
def find_best_recall(gridsearch):
    gridsearch.fit(X_train, y_train)
    print('Best score:', gridsearch.best_score_)
    print('Best parameters:', gridsearch.best_params_)
    print('Best estimator:', gridsearch.best_estimator_)
    return gridsearch.best_estimator_

In [110]:
pipe_etclf = pipe_maker(RandomForestClassifier(),)
params_etclf = {'clf__n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                'clf__criterion': ['gini', 'entropy'],
                'clf__max_features': ['auto', 'sqrt', 'log2'],
                'clf__max_depth': [None, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]}

In [111]:
find_best_recall(gridsearch_maker(pipe_etclf, params_etclf))

Best score: 0.7380251973100013
Best parameters: {'clf__criterion': 'entropy', 'clf__max_depth': 40, 'clf__max_features': 'sqrt', 'clf__n_estimators': 30}
Best estimator: Pipeline(steps=[('clf',
                 RandomForestClassifier(criterion='entropy', max_depth=40,
                                        max_features='sqrt',
                                        n_estimators=30))])


Pipeline(steps=[('clf',
                 RandomForestClassifier(criterion='entropy', max_depth=40,
                                        max_features='sqrt',
                                        n_estimators=30))])

In [114]:
clf = RandomForestClassifier(n_estimators=30, criterion='entropy', max_features='sqrt', max_depth=40)

In [115]:
clf.fit(X_train, y_train)
print('Training set score: {:.3f}'.format(clf.score(X_train, y_train)))
print('Test set score: {:.3f}'.format(clf.score(X_test, y_test)))
print(classification_report(y_test, clf.predict(X_test)))
print(confusion_matrix(y_test, clf.predict(X_test)))

Training set score: 0.980
Test set score: 0.759
              precision    recall  f1-score   support

           0       0.72      1.00      0.84        86
           1       1.00      0.35      0.52        51

    accuracy                           0.76       137
   macro avg       0.86      0.68      0.68       137
weighted avg       0.83      0.76      0.72       137

[[86  0]
 [33 18]]
